In [4]:
# example of training a stable gan for generating a handwritten digit
import tensorflow as tf
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import RandomNormal
from matplotlib import pyplot
 
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
# weight initialization
 init = RandomNormal(stddev=0.02)
# define model
 model = Sequential()
# downsample to 14x14
 model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
 model.add(LeakyReLU(alpha=0.2))
# downsample to 7x7
 model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
 model.add(LeakyReLU(alpha=0.2))
# classifier
 model.add(Flatten())
 model.add(Dense(1, activation='sigmoid'))
# compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model
 
# define the standalone generator model
def define_generator(latent_dim):
# weight initialization
 init = RandomNormal(stddev=0.02)
# define model
 model = Sequential()
# foundation for 7x7 image
 n_nodes = 128 * 7 * 7
 model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Reshape((7, 7, 128)))
# upsample to 14x14
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
 model.add(LeakyReLU(alpha=0.2))
# upsample to 28x28
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
 model.add(LeakyReLU(alpha=0.2))
# output 28x28x1
 model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
 return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model
 
# load mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# select all of the examples for a given class
	selected_ix = trainy == 8
	X = X[selected_ix]
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y
 
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	pyplot.savefig('results_opt/generated_plot_%03d.png' % (step+1))
	pyplot.close()
	# save the generator model
	g_model.save('results_opt/model_%03d.h5' % (step+1))
 
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a1_hist, label='acc-real')
	pyplot.plot(a2_hist, label='acc-fake')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('results_opt/plot_line_plot_loss.png')
	pyplot.close()
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
	plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)
 
# make folder for results
makedirs('results_opt', exist_ok=True)
# size of the latent space
latent_dim = 50
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

(5851, 28, 28, 1)
>1, d1=0.693, d2=0.699 g=0.689, a1=48, a2=0
>2, d1=0.505, d2=0.733 g=0.649, a1=100, a2=0
>3, d1=0.334, d2=0.889 g=0.537, a1=100, a2=0
>4, d1=0.221, d2=1.181 g=0.426, a1=100, a2=0
>5, d1=0.188, d2=1.195 g=0.454, a1=100, a2=0
>6, d1=0.209, d2=0.937 g=0.605, a1=100, a2=0
>7, d1=0.240, d2=0.703 g=0.796, a1=100, a2=0
>8, d1=0.290, d2=0.531 g=1.010, a1=100, a2=100
>9, d1=0.245, d2=0.394 g=1.288, a1=100, a2=100
>10, d1=0.258, d2=0.270 g=1.665, a1=96, a2=100
>11, d1=0.221, d2=0.167 g=2.153, a1=92, a2=100
>12, d1=0.210, d2=0.095 g=2.711, a1=95, a2=100
>13, d1=0.238, d2=0.054 g=3.223, a1=90, a2=100
>14, d1=0.116, d2=0.032 g=3.701, a1=100, a2=100
>15, d1=0.141, d2=0.021 g=4.060, a1=92, a2=100
>16, d1=0.167, d2=0.016 g=4.207, a1=90, a2=100
>17, d1=0.066, d2=0.015 g=4.276, a1=100, a2=100
>18, d1=0.032, d2=0.014 g=4.325, a1=100, a2=100
>19, d1=0.031, d2=0.013 g=4.354, a1=100, a2=100
>20, d1=0.062, d2=0.014 g=4.302, a1=96, a2=100
>21, d1=0.031, d2=0.014 g=4.240, a1=98, a2=100
>22, d

>171, d1=0.000, d2=0.000 g=8.112, a1=100, a2=100
>172, d1=0.003, d2=0.000 g=7.485, a1=100, a2=100
>173, d1=0.000, d2=0.001 g=7.113, a1=100, a2=100
>174, d1=0.000, d2=0.001 g=7.024, a1=100, a2=100
>175, d1=0.000, d2=0.001 g=7.181, a1=100, a2=100
>176, d1=0.000, d2=0.001 g=7.512, a1=100, a2=100
>177, d1=0.000, d2=0.000 g=7.927, a1=100, a2=100
>178, d1=0.000, d2=0.000 g=8.221, a1=100, a2=100
>179, d1=0.000, d2=0.000 g=8.383, a1=100, a2=100
>180, d1=0.000, d2=0.000 g=8.530, a1=100, a2=100
>181, d1=0.000, d2=0.000 g=8.649, a1=100, a2=100
>182, d1=0.000, d2=0.000 g=8.747, a1=100, a2=100
>183, d1=0.001, d2=0.000 g=8.761, a1=100, a2=100
>184, d1=0.000, d2=0.000 g=8.781, a1=100, a2=100
>185, d1=0.000, d2=0.000 g=8.804, a1=100, a2=100
>186, d1=0.000, d2=0.000 g=8.830, a1=100, a2=100
>187, d1=0.000, d2=0.000 g=8.857, a1=100, a2=100
>188, d1=0.000, d2=0.000 g=8.888, a1=100, a2=100
>189, d1=0.000, d2=0.000 g=8.918, a1=100, a2=100
>190, d1=0.000, d2=0.000 g=8.944, a1=100, a2=100
>191, d1=0.000, d2=0

>339, d1=0.000, d2=0.000 g=9.645, a1=100, a2=100
>340, d1=0.000, d2=0.000 g=9.699, a1=100, a2=100
>341, d1=0.000, d2=0.000 g=9.744, a1=100, a2=100
>342, d1=0.000, d2=0.000 g=9.785, a1=100, a2=100
>343, d1=0.000, d2=0.000 g=9.823, a1=100, a2=100
>344, d1=0.000, d2=0.000 g=9.857, a1=100, a2=100
>345, d1=0.000, d2=0.000 g=9.889, a1=100, a2=100
>346, d1=0.000, d2=0.000 g=9.918, a1=100, a2=100
>347, d1=0.000, d2=0.000 g=9.945, a1=100, a2=100
>348, d1=0.000, d2=0.000 g=9.969, a1=100, a2=100
>349, d1=0.000, d2=0.000 g=9.993, a1=100, a2=100
>350, d1=0.000, d2=0.000 g=10.015, a1=100, a2=100
>351, d1=0.000, d2=0.000 g=10.037, a1=100, a2=100
>352, d1=0.000, d2=0.000 g=10.058, a1=100, a2=100
>353, d1=0.000, d2=0.000 g=10.078, a1=100, a2=100
>354, d1=0.000, d2=0.000 g=10.098, a1=100, a2=100
>355, d1=0.000, d2=0.000 g=10.107, a1=100, a2=100
>356, d1=0.000, d2=0.000 g=10.118, a1=100, a2=100
>357, d1=0.000, d2=0.000 g=10.130, a1=100, a2=100
>358, d1=0.000, d2=0.000 g=10.142, a1=100, a2=100
>359, d1=0.